# Temperature, salinity and surface forcing at ocean station Papa

This notebook converts the temperature and salinity profiles and the surface forcing data at the ocean station Papa from `netCDF` to `.dat` files. The `netCDF` data is obtained from the [NOAA PMEL](https://www.pmel.noaa.gov/ocs/Papa) [data archive](https://www.pmel.noaa.gov/ocs/data/disdel/).

In [ ]:
import sys
import os
import numpy as np
import xarray as xr
import pandas as pd
import xml.etree.ElementTree as et
sys.path.append("../../../gotmtool")
from gotmtool import *

### Preprocess data

Select a subset of the data

In [ ]:
paparoot = os.environ['HOME']+'/data/OCS/Papa/Papa_20100616-20150101'
startdate = '2012-03-20'
enddate = '2013-03-22'

File `OSPapa.xml` contains the information of different variables for conversion. Loop over all vairables, process and save the data.

In [ ]:
xmlfile = 'OSPapa.xml'
tree = et.parse(xmlfile)
root = tree.getroot()
# loop over all variables
for child in root:
    # prepare input data
    var   = child.attrib['name']
    fname = child.find('filename').text
    vtype = child.find('vartype').text
    oname = child.find('outname').text
    ds = xr.load_dataset(paparoot+'/'+fname).sel(time=slice(startdate, enddate))
    time = ds.time.values
    ntime = time.size
    dttime = [pd.Timestamp(time[i]).to_pydatetime() for i in np.arange(ntime)]
    depth = -np.abs(ds.depth.values)
    ndepth = depth.size
    if '_FillValue' in ds.attrs:
        skipvalue = ds.attrs['_FillValue']
    elif 'missing_value' in ds.attrs:
        skipvalue = ds.attrs['missing_value']
    else:
        skipvalue = None
    if vtype == 'scalar':
        # scalar variable
        vname = child.find('varname').text
        data = ds.data_vars[vname].values.squeeze()
        if ndepth == 1:
            dat_dump_ts(dttime, [data], oname, skip_value=skipvalue)
        else:
            dat_dump_pfl(dttime, depth, [data], oname, skip_value=skipvalue)
    elif vtype == 'vector':
        # vector variable
        vnamex = child.find('varnamex').text
        vnamey = child.find('varnamey').text
        datax = ds.data_vars[vnamex].values.squeeze()
        datay = ds.data_vars[vnamey].values.squeeze()
        if ndepth == 1:
            dat_dump_ts(dttime, [datax, datay], oname, skip_value=skipvalue)
        else:
            dat_dump_pfl(dttime, depth, [datax, datay], oname, skip_value=skipvalue)